In [1]:
import numpy as np
import pandas as pd
import pdc_dtf
import sys
sys.path.append('/projectnb/cs542/wchapman/seizure_prediction_ml')
import utils

In [114]:
pat = 3
df = pd.read_pickle('/projectnb/cs542/wchapman/seizure_prediction_ml/arima/fits/s' +str(pat) +'_coefs')
#npz = np.load('/projectnb/cs542/wchapman/seizure_prediction_ml/arima/fits/s2_40Hz.npz')
dfs = utils.read_frames()
dfs = dfs[(dfs['pat']==pat) & (dfs['train']==1)]
y = dfs['class'].values

In [115]:
# unpack the variables
coef = np.zeros((df.shape[0], 
                 df.loc[0]['coef'][0].shape[0], 
                 df.loc[0]['coef'][0].shape[1],
                 df.loc[0]['coef'][0].shape[2]))

coef_block = np.zeros((df.shape[0], 
                       df.loc[0]['coef_block'][0].shape[0], 
                       df.loc[0]['coef_block'][0].shape[1],
                       df.loc[0]['coef_block'][0].shape[2],
                       df.loc[0]['coef_block'][0].shape[3]))

coef_flat = np.zeros((coef.shape[0],coef.shape[1]*coef.shape[2]*coef.shape[3]))
coef_flat_block = np.zeros((coef.shape[0], coef.shape[1], coef.shape[2]*coef.shape[3]))
y_blocks = np.zeros((coef.shape[0], coef.shape[1]))

for i in range(0, df.shape[0]):
    coef[i] = df.loc[i].coef[0]
    coef_block[i] = df.loc[i].coef_block[0]
    
    coef_flat[i] = coef[i].flatten()
    
    for k in range(0, coef[i].shape[0]):
        coef_flat_block[i][k] = coef[i][k].flatten()
        y_blocks[i] = y[i]

In [116]:
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score


rkf = StratifiedKFold(n_splits=10)
auc_trains = list()
auc_tests = list()

for trains, tests in rkf.split(coef_flat, y):
    model = LogisticRegression()
    model.fit(coef_flat[trains], y[trains])
    
    auc_train = (roc_auc_score(y[trains], model.predict_proba(coef_flat[trains])[:,1]))
    auc_test = (roc_auc_score(y[tests], model.predict_proba(coef_flat[tests])[:,1]))
 
    auc_trains.append(auc_train)
    auc_tests.append(auc_test)


In [120]:
print(np.mean(np.asarray(auc_tests)))
print(np.std(np.asarray(auc_tests)))

0.7994034026866107
0.11662706634807415


In [24]:
# %% Get ROC
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
logit_roc_auc = roc_auc_score(y_val, model.predict(coefv_flat))
fpr, tpr, thresholds = roc_curve(y_val, model.predict_prob(coefv_flat)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
#plt.savefig('Log_ROC')
plt.show()

NameError: name 'y_val' is not defined